In [2]:
# STEP 1: Install libraries
!pip install pandas python-docx

# STEP 2: Upload XLSForm
from google.colab import files
uploaded = files.upload()

import pandas as pd
from docx import Document
from docx.shared import RGBColor, Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import datetime
import os

# STEP 3: Load and clean Excel
xls_filename = list(uploaded.keys())[0]
survey = pd.read_excel(xls_filename, sheet_name='survey').fillna("").astype(str)
choices = pd.read_excel(xls_filename, sheet_name='choices').fillna("").astype(str)

# STEP 4: Detect language columns
label_columns = [col for col in survey.columns if col.startswith("label::")]
languages = [col.split("::")[1] for col in label_columns]

# STEP 5: Generate Word document for each language
for lang in languages:
    doc = Document()

    # Page setup
    section = doc.sections[0]
    section.page_height = Inches(11.69)
    section.page_width = Inches(8.27)
    section.top_margin = Inches(0.75)
    section.bottom_margin = Inches(0.5)
    section.left_margin = Inches(0.5)
    section.right_margin = Inches(0.5)

    # Font style
    style = doc.styles['Normal']
    style.font.name = 'Calibri Light'
    style.font.size = Pt(9)

    # Title
    form_title = os.path.splitext(xls_filename)[0]
    title_para = doc.add_paragraph()
    title_run = title_para.add_run(f"{form_title} ({lang})")
    title_run.bold = True
    title_run.font.size = Pt(18)
    title_run.font.color.rgb = RGBColor(0, 128, 0)
    title_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Date
    today = datetime.datetime.today().strftime('%d %B %Y')
    date_para = doc.add_paragraph()
    date_run = date_para.add_run(f"Generated on: {today}")
    date_run.font.color.rgb = RGBColor(128, 128, 128)
    date_run.font.size = Pt(14)
    date_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Credits
    credit_para = doc.add_paragraph()
    credit_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    credit_name = credit_para.add_run("Md. Zahirul Islam\n")
    credit_name.bold = True
    credit_name.font.size = Pt(10)
    credit_role = credit_para.add_run("Senior Project Associate, ARCED Foundation\n")
    credit_role.font.size = Pt(9)
    credit_contact = credit_para.add_run("Email: zahirul.islam@arced.foundation, Cell: +8801688831919")
    credit_contact.font.size = Pt(9)

    doc.add_paragraph()  # spacer

    # Create table
    table = doc.add_table(rows=1, cols=5)
    table.style = 'Table Grid'
    headers = ['Variable', 'Question', 'Choices', 'Constraint', 'Relevance']
    for i, h in enumerate(headers):
        hdr_run = table.rows[0].cells[i].paragraphs[0].add_run(h)
        hdr_run.bold = True
        hdr_run.font.name = 'Calibri Light'
        hdr_run.font.size = Pt(9)

    def set_repeat_table_header(row):
        tr = row._tr
        trPr = tr.get_or_add_trPr()
        tblHeader = OxmlElement('w:tblHeader')
        tblHeader.set(qn('w:val'), "true")
        trPr.append(tblHeader)

    set_repeat_table_header(table.rows[0])

    # Build language-specific choices_dict
    choices_dict = {}
    choice_label_col = f"label::{lang}"
    for _, row in choices.iterrows():
        list_name = row.get('list_name', '').strip()
        value = row.get('value', '').strip()
        label = row.get(choice_label_col, '').strip()
        if list_name and value and label:
            if list_name not in choices_dict:
                choices_dict[list_name] = []
            choices_dict[list_name].append(f"[{value}] {label}")

    # Loop through survey
    for _, row in survey.iterrows():
        qtype = row.get('type', '').strip()
        name = row.get('name', '').strip()
        label = row.get(f'label::{lang}', '').strip()
        constraint = row.get('constraint', '').strip()
        relevance = row.get('relevance', '').strip()

        if not any([name, label, qtype]):
            continue

        # Determine choices
        choices_text = ""
        if qtype.startswith('select_one') or qtype.startswith('select_multiple'):
            parts = qtype.split()
            if len(parts) > 1:
                list_name = parts[1].strip()
                choices_text = "\n".join(choices_dict.get(list_name, ["⚠️ Choices not found"]))
            else:
                choices_text = "⚠️ list_name missing"
        elif 'note' in qtype.lower():
            choices_text = "[Enumerator Note]"
        elif 'text' in qtype.lower():
            choices_text = "[Text]"
        elif 'date' in qtype.lower():
            choices_text = "[Date]"
        elif 'integer' in qtype.lower():
            choices_text = "[Number]"
        elif 'geopoint' in qtype.lower() or 'gps' in name.lower():
            choices_text = "[Lat, Long, Alt]"

        # Add row
        row_cells = table.add_row().cells

        var_run = row_cells[0].paragraphs[0].add_run(name)
        var_run.italic = True
        var_run.font.color.rgb = RGBColor(0, 128, 0)
        var_run.font.size = Pt(9)
        var_run.font.name = 'Calibri Light'

        q_run = row_cells[1].paragraphs[0].add_run(label)
        q_run.font.size = Pt(9)
        q_run.font.name = 'Calibri Light'

        ch_run = row_cells[2].paragraphs[0].add_run(choices_text)
        ch_run.font.size = Pt(9)
        ch_run.font.name = 'Calibri Light'

        c_run = row_cells[3].paragraphs[0].add_run(constraint)
        c_run.font.size = Pt(9)
        c_run.font.name = 'Calibri Light'

        r_run = row_cells[4].paragraphs[0].add_run(relevance)
        r_run.font.size = Pt(9)
        r_run.font.name = 'Calibri Light'

    # Footer
    footer_para = section.footer.paragraphs[0]
    footer_para.clear()
    footer_para.paragraph_format.tab_stops.add_tab_stop(Inches(6.5))

    left_run = footer_para.add_run("Generated by: Md. Zahirul Islam")
    left_run.font.name = 'Calibri Light'
    left_run.font.size = Pt(9)
    left_run.italic = True

    footer_para.add_run("\t")
    right_run = footer_para.add_run("Page")
    right_run.font.name = 'Calibri Light'
    right_run.font.size = Pt(9)
    right_run.italic = True

    footer_para.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    # Save Word file
    output_filename = f"{form_title}_{lang}_word.docx"
    doc.save(output_filename)
    files.download(output_filename)


Saving BARD_survey_25_Rasel.xlsx to BARD_survey_25_Rasel (1).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>